In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import shutil

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql import Window

In [3]:
test_path = "../data/test_users.json"
ratings_path = "../data/base_rating_prq"

In [4]:
spark = SparkSession.builder \
            .appName("ALSbuilder") \
            .getOrCreate()

In [8]:
rating_df = spark.read.parquet(ratings_path)

In [9]:
rating_df.show()

+-----------+--------+------------------+-------------------+
|element_uid|user_uid|                ts|    true_watch_part|
+-----------+--------+------------------+-------------------+
|       7642|  398055| 42890955.82399276| 1.1703333333333332|
|        775|  312530|42948120.417987145| 0.3293333333333333|
|       9742|  459840| 44202466.49127187|0.06555555555555556|
|       2694|  366950| 42078259.94467698| 1.1810833333333333|
|      10061|  460080| 42566616.78668126| 0.9904999999999999|
|       6432|  280809| 42103269.00310552| 0.8261666666666666|
|        283|  117180| 42471787.11508496| 0.8853333333333333|
|       8863|  433042| 43128230.36225684|   1.16929012345679|
|       6728|  524287| 43893810.59373309|             0.0075|
|       2024|  495586| 43821190.91957284|  1.170534188034188|
|       6872|   95978|42310750.323095135|            1.20625|
|       7150|  300940| 42638277.56487383| 1.1723484848484849|
|       2567|  143250|  41766370.9448332| 0.4873809523809524|
|       

In [10]:
#Заменяем имена колонок на стандарные и корректируем типы данных

userCol = 'user_uid'
itemCol = 'element_uid'
ratingCol = 'true_watch_part'


rating_df = rating_df.withColumnRenamed(itemCol, 'item_id')\
        .withColumnRenamed(ratingCol, 'rate')\
        .withColumnRenamed(userCol, 'user_id')\
        .withColumn("user_id", col("user_id").cast('int'))\
        .withColumn("item_id", col("item_id").cast('int'))

In [11]:
#Убираем пользователей, которые посмотрели меньше 3х фильмов

film_cnt = rating_df.groupBy('user_id').count()\
            .withColumn('enough_films', col('count') >= 3)

rating_df = rating_df.join(film_cnt, on='user_id', how='left')\
            .where(col('enough_films') == True)

### OOT split

OOT разбиение на train и test (сделать ячейку активной, если необходимо использовать его)

### randomSplit 

Случайное разбиение на train и test (сделать ячейку активной, если необходимо использовать его)

In [12]:
rating_df = rating_df.drop("ts").limit(10000)

train_als, train_cb, test = rating_df.randomSplit([0.4, 0.4, 0.2])

### Client oriented split on train/train/test

Разбиение на train и test oot по каждому клиенту (сделать ячейку активной, если необходимо использовать его)

### Dump train_als/train_cb/test

In [16]:
def write_parquet(df, path):
    if os.path.exists(path):
        shutil.rmtree(path)
        df.write.parquet(path)
    else:
        df.write.parquet(path)

In [17]:
write_parquet(train_als, "../data/train_als")
write_parquet(train_cb, "../data/train_cb")
write_parquet(test, "../data/test")

### First level model building

In [19]:
train_als = spark.read.parquet("../data/train_als")

In [20]:
als = ALS(maxIter=10, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rate",
          coldStartStrategy="drop", implicitPrefs=True)

model = als.fit(train_als)

#TODO подбор параметров

In [21]:
@udf(returnType=ArrayType(IntegerType()))
def get_film_ids(arr):
    """
    Функция для извлечения id фильмов после предикта ALS
    """
    return [x[0] for x in arr]

### Test on boosters

Для тестирования ALS на Boosters.pro

### Create predictions

In [22]:
rec_df = model.recommendForAllUsers(100)

In [23]:
%%time
rec_df = rec_df.repartition(500)

CPU times: user 969 µs, sys: 1.36 ms, total: 2.33 ms
Wall time: 5.57 ms


In [26]:
%%time
prediction_path = '../data/first_level_output_prq'
write_parquet(rec_df, prediction_path)

CPU times: user 3 ms, sys: 7.25 ms, total: 10.2 ms
Wall time: 20 s
